# Web Scraping For Data Unification.
<a id='top_cell'></a>
Using the datasets from http://medata.gov.co/node/85 we use web scraping to unify the datasets on two sets, Resources and Crime.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import frameParser as parser

Capture the information of the API provided, and generate a dictionary the names and each download link

In [ ]:
firefox_driver_path = "/home/felipe/DS4A/Project/code/datasets_scraping/geckodriver"

In [ ]:
driver = webdriver.Firefox(executable_path=firefox_driver_path)
button = "//*[@id='data-and-resources']/div/div/ul/li/div/span/a[2]"
sources = {}
for k in range(0, 4):
    driver.get(
        f"http://medata.gov.co/node/85?search_api_views_fulltext=&sort_by=title&sort_order=ASC&q=node/85&page={k}")

    for i in range(1, 11):
        element = driver.find_element(By.XPATH,
                                      f"//*[@id='main']/div/section/div/div/div/div/div[2]/div/div[2]/div/div/div[2]/div[{i}]/article/div[2]/h2/a")
        name = element.text
        element.click()
        href = driver.find_element(By.XPATH,
                                   "/html/body/div[2]/div/div/section/div/div/div/div/div[2]/div/div/div/article/div/div[4]/div/div/ul/li/div/span/a[2]").get_attribute(
            'href')
        print(f"{k},{i} {name}: {href}")
        sources[name] = href
        driver.back()

/tmp/ipykernel_103321/3979411052.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=chrome_driver_path)


0,1 Aprehensión: http://medata.gov.co/node/23760/download
0,2 Captura: http://medata.gov.co/node/23762/download
0,3 Conducción para el restablecimiento de derechos: http://medata.gov.co/node/23745/download
0,4 Conducta contraria a la convivencia: http://medata.gov.co/node/23767/download
0,5 Consolidado cantidad de casos criminalidad en comunas por año: http://medata.gov.co/node/24152/download
0,6 Consolidado cantidad de casos criminalidad en comunas por año y mes: http://medata.gov.co/node/24153/download
0,7 Consolidado cantidad de casos criminalidad por año: http://medata.gov.co/node/24150/download
0,8 Consolidado cantidad de casos criminalidad por año y mes: http://medata.gov.co/node/24151/download
0,9 Extorsión: http://medata.gov.co/node/23747/download
0,10 Homicidio: http://medata.gov.co/node/23751/download
1,1 Hurto a entidad financiera: http://medata.gov.co/node/23757/download
1,2 Hurto a establecimiento comercial: http://medata.gov.co/node/23763/download
1,3 Hurto a persona: htt

NoSuchElementException: Message: Unable to locate element: //*[@id='main']/div/section/div/div/div/div/div[2]/div/div[2]/div/div/div[2]/div[5]/article/div[2]/h2/a
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:395:5
element.find/</<@chrome://remote/content/marionette/element.js:300:16


make the dictionary a dataframe

In [ ]:
#pd.DataFrame(sources.items())
data = pd.DataFrame(columns=parser.cols)
for i, source in enumerate(sources):
    print(f"{i + 1},{source}")  #{sources[source]}")
    df = parser.get_df_from_url(sources[source])
    #data = parser.concat_df_info(df, source, data)

1,Aprehensión
2,Captura
3,Conducción para el restablecimiento de derechos
4,Conducta contraria a la convivencia
5,Consolidado cantidad de casos criminalidad en comunas por año
6,Consolidado cantidad de casos criminalidad en comunas por año y mes
7,Consolidado cantidad de casos criminalidad por año
8,Consolidado cantidad de casos criminalidad por año y mes
9,Extorsión
10,Homicidio
11,Hurto a entidad financiera
12,Hurto a establecimiento comercial
13,Hurto a persona
14,Hurto a personas en transporte público
15,Hurto a residencia
16,Hurto de carro
17,Hurto de moto
18,Hurto de semoviente
19,Hurto por piratería terrestre
20,Incautación arma de fuego
21,Incautación mercancía
22,Lesión fatal accidente de tránsito según policía
23,Lesión fatal accidente de tránsito según policía con precipitación
24,Lesión no fatal accidental según policía
25,Lesión no fatal accidente de tránsito según policía
26,Lesión no fatal dolosa según policía
27,Ley1098
28,Llamada nuse123
29,Recuperación de automotores


We generate de united dataframes and save it

In [ ]:
num_dic = {}
for i, k in enumerate(data.dataframe.unique()):
    num_dic[k] = i + 1

data['df_num'] = data['dataframe'].map(num_dic)
data = data.set_index('df_num')
data

,dataframe,col_num,tipo,nombre,total,registros,num_categorias,categorias_unicas,desc_info
df_num,,,,,,,,,
1,Aprehensión,1,fecha,fecha_hecho,1043,1043,13,"2004: 137, 2005: 163, 2006: 223, 2007: 381, 20...","count: 1043, unique: 984, top: 2005-07-11 11:5..."
1,Aprehensión,2,float64,cantidad,1043,1043,1,"1.0: 1043,","count: 1043.0, mean: 1.0, std: 0.0, min: 1.0, ..."
1,Aprehensión,3,float64,latitud,1043,808,487,mas de 200,"count: 808.0, mean: 6.253193913737623, std: 0...."
1,Aprehensión,4,float64,longitud,1043,808,484,mas de 200,"count: 808.0, mean: -75.5686218609406, std: 0...."
1,Aprehensión,5,object,sexo,1043,1043,3,"Hombre: 961, Mujer: 76, Sin dato: 6,","count: 1043, unique: 3, top: Hombre, freq: 961,"
...,...,...,...,...,...,...,...,...,...
34,Solicitud de medidas de protección por violenc...,32,object,modelo,83428,83428,1,"Sin dato: 83428,","count: 83428, unique: 1, top: Sin dato, freq: ..."
34,Solicitud de medidas de protección por violenc...,33,object,color,83428,83428,1,"Sin dato: 83428,","count: 83428, unique: 1, top: Sin dato, freq: ..."
34,Solicitud de medidas de protección por violenc...,34,object,permiso,83428,83428,1,"Sin dato: 83428,","count: 83428, unique: 1, top: Sin dato, freq: ..."


## Dictionary with the categories
We generate a Dictionary that contain each dataset name and each final group dataset so we could divide on Resources (Recursos) or Crime (Crimen) datasets.

In [ ]:
df_dict = {
    "Aprehensión": "Recurso",
    "Captura": "Recurso",
    "Conducción para el restablecimiento de derechos": "Recurso",
    "Conducta contraria a la convivencia": "Recurso",
    "Extorsión": "Crimen",
    "Homicidio": "Crimen",
    "Hurto a entidad financiera": "Crimen",
    "Hurto a establecimiento comercial": "Crimen",
    "Hurto a persona": "Crimen",
    "Hurto a personas en transporte público": "Crimen",
    "Hurto a residencia": "Crimen",
    "Hurto de carro": "Crimen",
    "Hurto de moto": "Crimen",
    "Hurto de semoviente": "Crimen",
    "Hurto por piratería terrestre": "Crimen",
    "Incautación arma de fuego": "Recurso",
    "Incautación mercancía": "Recurso",
    "Lesión fatal accidente de tránsito según policía": "Recurso",
    "Lesión fatal accidente de tránsito según policía con precipitación": "Recurso",
    "Lesión no fatal accidental según policía": "Recurso",
    "Lesión no fatal accidente de tránsito según policía": "Recurso",
    "Lesión no fatal dolosa según policía": "Recurso",
    "Ley1098": "Recurso",
    "Llamada nuse123": "Recurso",
    "Recuperación de automotores": "Recurso",
    "Recuperación de mercancía": "Recurso",
    "Reincidencia violencia intrafamiliar": "Crimen",
    "Secuestro": "Crimen",
    "Solicitud de medidas de protección por violencia intrafamiliar": "Recurso",
}

In [ ]:
df_crime = pd.DataFrame()
df_resource = pd.DataFrame()
for i, source in enumerate(sources):
    if source in df_dict:
        print(f"{i + 1},{source}, {df_dict[source]}")  #{sources[source]}")
        df = parser.get_df_from_url(sources[source])
        df['Nbase'] = source
        print('loaded')
        df.set_index('Nbase')
        if df_dict[source] == 'Crimen':
            df_crime = df_crime.append(df)
        else:
            df_resource = df_resource.append(df)

1,Aprehensión, Recurso
loaded
2,Captura, Recurso


/tmp/ipykernel_103321/901921535.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_resource = df_resource.append(df)
/home/felipe/DS4A/Project/code/datasets_scraping/frameParser.py:12: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(io.StringIO(content.decode('utf-8')), sep=";")


loaded


/tmp/ipykernel_103321/901921535.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_resource = df_resource.append(df)


3,Conducción para el restablecimiento de derechos, Recurso
loaded
4,Conducta contraria a la convivencia, Recurso


...

/tmp/ipykernel_103321/901921535.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_crime = df_crime.append(df)


loaded
34,Solicitud de medidas de protección por violencia intrafamiliar, Recurso


/tmp/ipykernel_103321/901921535.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_crime = df_crime.append(df)


loaded


/tmp/ipykernel_103321/901921535.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_resource = df_resource.append(df)


In [ ]:
df_crime.to_csv('dataframes/df_crimenes.csv', index=False)

In [ ]:
df_resource.to_csv('dataframes/df_recursos.csv', index=False)